### Importing libraries and reading the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('flight_price.csv')
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


### Preliminary Analysis and Checking

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10,683 rows and 11 features
- Columns `Route` and `Total_Stops` have missing value each
- The dataset has only one integer column `Price` which will be the target variable and 10 object columns

In [4]:
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


- Maximum flight price is 79512 and minimum flight price is 1759 making an average of 9087

In [5]:
df.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [6]:
df.duplicated().sum()

220

In [7]:
( df.loc[df.duplicated(keep=False)]
 .sort_values(['Airline', 'Date_of_Journey', 'Source', 'Destination'])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


##### Observations
- Type of` Date_of_Journey`, `Dep_Time` and `Arrival_Time` should be changed to datetime
- Type of `Duration` and `Total_Stops` is mixed. It should be numeric type
- **220** duplicate values are present which should be removed.

### Detailed Analysis by observing each column

In [8]:
df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [9]:
df.Airline.str.replace(" Premium economy", "").str.replace(" Business", "").str.title()

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [10]:
pd.to_datetime(df.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

In [11]:
pd.to_datetime(df.Dep_Time).dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [12]:
df.Dep_Time.loc[lambda ser:ser.str.contains("^[0-9]:")]

Series([], Name: Dep_Time, dtype: object)

In [13]:
pd.to_datetime(df.Arrival_Time).dt.time

0        01:10:00
1        13:15:00
2        04:25:00
3        23:30:00
4        21:35:00
           ...   
10678    22:25:00
10679    23:20:00
10680    11:20:00
10681    14:10:00
10682    19:15:00
Name: Arrival_Time, Length: 10683, dtype: object

In [14]:
df.Arrival_Time.loc[lambda ser: ser.str.contains("[^0-9:]")].str.split(" ", n=1).str.get(1).unique()

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

In [15]:
df.Duration.loc[lambda ser: ~ser.str.contains("m")].unique()

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [16]:
df.Duration.loc[lambda ser: ~ser.str.contains("h")]

6474    5m
Name: Duration, dtype: object

- The observation indexed 6474 has duration of 5 minutes which is clearly wrong. So lets's delete this observation.

In [17]:
df.drop(index=6474)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [18]:
( df.Duration.drop(index=6474).str.split(" ", expand=True).set_axis(["hour", "minute"], axis=1)
     .assign( hour=lambda df_: (df_.hour.str.replace("h", "").astype(int).mul(60)),
            minute=lambda df_: (df_.minute.str.replace("m", "").fillna("0").astype(int))
            ).sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [19]:
( df.Duration.drop(index=6474).str.split(" ", expand=True).set_axis(["hour", "minute"], axis=1)
     .assign( hour=lambda df_: (df_.hour.str.replace("h", "").astype(int).mul(60)),
            minute=lambda df_: (df_.minute.str.replace("m", "").fillna("0").astype(int))
            ).sum(axis=1).rename("duration_minutes").to_frame())# .join(df.Duration.drop(index=6474)))

,duration_minutes
0,170
1,445
2,1140
3,325
4,285
...,...
10678,150
10679,155
10680,180
10681,160


In [20]:
def convert_to_minutes(ser):
    return ( df.Duration.str.split(" ", expand=True).set_axis(["hour", "minute"], axis=1)
             .assign( hour=lambda df_: (df_.hour.str.replace("h", "").astype(int)*60),
              minute=lambda df_: (df_.minute.str.replace("m", "").fillna("0").astype(int))
              ).sum(axis=1)
           )

In [21]:
df.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [22]:
df.Total_Stops.replace('non-stop','0').str.replace(' stops?','',regex=True).fillna('0').astype(int).to_frame()

,Total_Stops
0,0
1,2
2,2
3,1
4,1
...,...
10678,0
10679,0
10680,0
10681,0


In [23]:
df.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [24]:
df.Additional_Info.replace('No info','No Info').unique()

array(['No Info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [25]:
df.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [26]:
df.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

- Source and destination column look perfect, so we don't modify them

### Creating  functions to apply all the operations to clean the data

In [27]:
def convert_to_minutes(ser):
    return ( ser.str.split(" ", expand=True).set_axis(["hour", "minute"], axis=1)
             .assign( hour=lambda df_: (df_.hour.str.replace("h", "").astype(int).mul(60)),
              minute=lambda df_: (df_.minute.str.replace("m", "").fillna("0").astype(int))
              ).sum(axis=1)
           )
    

In [28]:
def clean_data(df):
    return ( df.drop(index=[6474]).drop_duplicates()
            .assign(**{col: df[col].str.strip()
                        for col in df.select_dtypes(include="O").columns})
            .rename(columns=str.lower)
            .assign(airline = lambda df_:df_.airline.str.replace(" Premium economy", "").str.replace(" Business", "").str.title(),
                     date_of_journey=lambda df_: pd.to_datetime(df_.date_of_journey, dayfirst=True),
                     dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
                     arrival_time=lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
                     duration=lambda df_: df_.duration.pipe(convert_to_minutes),
                     total_stops=lambda df_: (df_.total_stops.replace("non-stop", "0").str.replace(" stops?", "", regex=True)
                                        .fillna('0').astype(int)),
                     additional_info=lambda df_: df_.additional_info.replace("No info", "No Info")
                    )       
           .drop(columns='route')   
           )

In [29]:
cleaned_df = clean_data(df)
cleaned_df.head(6)

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1,No Info,13302
5,Spicejet,2019-06-24,Kolkata,Banglore,09:00:00,11:25:00,145,0,No Info,3873


In [30]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10462 entries, 0 to 10682
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   airline          10462 non-null  object        
 1   date_of_journey  10462 non-null  datetime64[ns]
 2   source           10462 non-null  object        
 3   destination      10462 non-null  object        
 4   dep_time         10462 non-null  object        
 5   arrival_time     10462 non-null  object        
 6   duration         10462 non-null  int64         
 7   total_stops      10462 non-null  int32         
 8   additional_info  10462 non-null  object        
 9   price            10462 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(2), object(6)
memory usage: 858.2+ KB


### Splitting the data

In [31]:
final_df = cleaned_df.sample(3000)

In [32]:
X = final_df.drop(columns='price')
y = final_df.price.copy()

In [33]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(1920, 9) (1920,)
(480, 9) (480,)
(600, 9) (600,)


### Exporting the Subsets

In [34]:
mkdir data

A subdirectory or file data already exists.


In [35]:
import os
def export_data(X, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join('data', file_name)
    X.join(y).to_csv(file_path, index=False)
    return pd.read_csv(file_path).head(6)


In [36]:
export_data(X_train,y_train,'tr')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Kolkata,Banglore,20:20:00,22:55:00,155,0,No Info,4804
1,Air India,2019-06-01,Delhi,Cochin,05:15:00,07:40:00,1585,3,No Info,12383
2,Jet Airways,2019-06-27,Delhi,Cochin,23:05:00,12:35:00,810,2,In-flight meal not included,11150
3,Jet Airways,2019-05-21,Kolkata,Banglore,20:25:00,14:25:00,1080,1,No Info,11467
4,Vistara,2019-05-15,Banglore,Delhi,19:30:00,22:15:00,165,0,No Info,4878
5,Jet Airways,2019-03-24,Mumbai,Hyderabad,10:20:00,11:50:00,90,0,In-flight meal not included,2759


In [37]:
export_data(X_val,y_val,'val')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air Asia,2019-03-27,Delhi,Cochin,16:45:00,22:15:00,330,1,No Info,6151
1,Vistara,2019-04-09,Banglore,Delhi,11:30:00,14:20:00,170,0,No Info,5403
2,Indigo,2019-03-06,Delhi,Cochin,07:30:00,15:30:00,480,1,No Info,14871
3,Spicejet,2019-06-12,Mumbai,Hyderabad,05:45:00,07:15:00,90,0,No Info,2753
4,Jet Airways,2019-05-01,Kolkata,Banglore,09:35:00,21:05:00,690,1,In-flight meal not included,12384
5,Indigo,2019-03-03,Delhi,Cochin,04:55:00,16:10:00,675,1,No Info,8513


In [38]:
export_data(X_test,y_test,'te')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-05-15,Kolkata,Banglore,16:30:00,10:05:00,1055,1,No Info,14781
1,Air India,2019-03-24,Kolkata,Banglore,20:30:00,08:55:00,745,1,No Info,12173
2,Multiple Carriers,2019-06-15,Delhi,Cochin,13:15:00,22:30:00,555,1,No Info,8018
3,Jet Airways,2019-06-24,Delhi,Cochin,02:15:00,12:35:00,620,1,No Info,18371
4,Multiple Carriers,2019-03-09,Delhi,Cochin,04:45:00,19:45:00,900,1,No Info,10782
5,Jet Airways,2019-03-06,Delhi,Cochin,23:05:00,19:45:00,1240,1,No Info,17024
